## Function Definitions, and mock function call results

In [43]:
import json
import uuid
from functools import partial


def add(args: str):
    args = json.loads(args)
    return str(float(args["a"]) + float(args["b"]))


def sub(args: str):
    args = json.loads(args)
    return str(float(args["a"]) - float(args["b"]))


def mult(args: str):
    args = json.loads(args)
    return str(float(args["a"]) * float(args["b"]))


def div(args: str):
    args = json.loads(args)
    return str(float(args["a"]) / float(args["b"]))


def get_oai_response(model, functions, msgs, api_key, base_url):
  import openai
  openai.api_key = api_key 
  openai.base_url = base_url
  
  try:
    completion = openai.chat.completions.create(
      model=model,
      temperature=0.1,
      messages=msgs,
      tools=functions,
      tool_choice="auto",
      stream=False,
    )
    return completion.choices[0]
  except Exception as e:
    print(e)


def insert_tool_response(res, msgs):
    assistant_message = res.message
    tool_calls = []
    for tool_call in assistant_message.tool_calls:
        tool_calls.append( {
                            "id": tool_call.id,
                            "function": {"name": tool_call.function.name,
                                        "arguments": tool_call.function.arguments},
                            "type": "function",
                        })
    msgs.append({"role": "assistant",  "tool_calls": tool_calls})
    
    for i, tool_call in enumerate(assistant_message.tool_calls):
        if tool_call.function.name == "getCurrentWeather":
            print()
            l = len((json.loads(assistant_message.tool_calls[i].function.arguments))["location"])
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"temprature is {(i+1) * 50 + l } degree"})
        elif tool_call.function.name == "calculate_distance":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Distance is {(i+1) * 50} miles."})
        elif tool_call.function.name == "generate_password":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Password generated: {uuid.uuid4().hex[:8]}"})
        elif tool_call.function.name == "orderUmbrella":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Order placed. the price is {(i+1) * 10} dollars."})
        elif tool_call.function.name == "list_files":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"File list:\nreport.docx\ntask.txt\nnotes.txt"})
        elif tool_call.function.name == "get_file_size":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"the size is {(i+1) * 100} bytes."})
        elif tool_call.function.name == "addition":
            msgs.append({
                "role": "tool",
                "name": "addition",
                "content": add(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "subtraction":
            msgs.append({
                "role": "tool",
                "name": "subtraction",
                "content": sub(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "multiplication":
            msgs.append({
                "role": "tool",
                "name": "multiplication",
                "content": mult(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "division":
            msgs.append({
                "role": "tool",
                "name": "division",
                "content": div(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        print(f"Observation: {msgs[-1]}")
    
    return msgs

def run_completion(chat_method, user_query, msgs=[]):
    system_prompt = "You are a helpful assistant."
    functions = [
        {
            "type": "function",
            "function": {
                "name": "list_files",
                "description": "List all files in a directory",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "directory": {
                            "type": "string",
                            "description": "the directory to list files from"
                        }
                    },
                    "required": [
                        "directory"
                    ]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "description": "Create a 3D model of an object with specified dimensions",
                "name": "create_3d_model",
                "parameters": {
                    "properties": {
                    "object_name": {
                        "description": "Name of the object to be modeled",
                        "type": "string"
                    },
                    "dimensions": {
                        "description": "Dimensions of the 3D object (length, width, height)",
                        "type": "object",
                        "properties": {
                        "length": {
                            "type": "number"
                        },
                        "width": {
                            "type": "number"
                        },
                        "height": {
                            "type": "number"
                        }
                        },
                        "required": [
                        "length",
                        "width",
                        "height"
                        ]
                    }
                    },
                    "required": [
                    "object_name",
                    "dimensions"
                    ],
                    "type": "object"
                }
            }
        },
        {
            "type": "function",
            "function": {
                "description": "Get the latest insurance premium from a list of premiums.",
                "name": "latest_insurance_premium",
                "parameters": {
                    "properties": {
                        "premiums": {
                            "description": "List of insurance premiums",
                            "type": "array",
                            "items": {
                            "type": "number"
                            }
                        }
                    },
                    "required": [
                        "premiums"
                    ],
                    "type": "object"
                }
            }
        },
        {
            "type": "function",
            "function": {
            "description": "Calculate insurance premium based on age and coverage",
            "name": "calculate_insurance_premium",
            "parameters": {
                "properties": {
                "age": {
                    "description": "Age of the person applying for insurance",
                    "type": "integer"
                },
                "coverage_type": {
                    "description": "Type of insurance coverage",
                    "type": "string",
                    "enum": [
                    "basic",
                    "standard",
                    "premium"
                    ]
                }
                },
                "required": [
                "age",
                "coverage_type"
                ],
                "type": "object"
            }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_file_size",
                "description": "Get the size of a file in bytes",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "filename": {
                            "type": "string",
                            "description": "the name of the file to get its size"
                        }
                    },
                    "required": [
                        "filename"
                    ]
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'addition',
                'description': "Adds two numbers together",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to add',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to add',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'subtraction',
                'description': "Subtracts two numbers",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to be subtracted from',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Number to subtract',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'multiplication',
                'description': "Multiply two numbers together",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to multiply',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to multiply',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'division',
                'description': "Divide two numbers",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to use as the dividend',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to use as the divisor',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
      {
          "type": "function",
        "function": {
          "name": "getCurrentWeather",
          "description": "Get the weather in location",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
              "unit": {"type": "string", "enum": ["c", "f"]}
            },
            "required": ["location"]
          }
        }
      },
      {    "type": "function",
        "function":
        {
            "name": "orderUmbrella",
            "description": "Do this to help user to order an umbrella online", 
            "parameters": {
                "type": "object",
                "properties": {
                    "number_to_buy": {
                        "type": "integer",
                        "description": "the amount of umbrellas to buy"
                    }
                },
                "required": [
                    "number_to_buy"
                ]
            }
        }},
      {"type": "function","function":{"name":"calculate_distance","description":"Calculate the distance between two locations","parameters":{"type":"object","properties":{"origin":{"type":"string","description":"The starting location"},"destination":{"type":"string","description":"The destination location"},"mode":{"type":"string","description":"The mode of transportation"}},"required":["origin","destination","mode"]}}},{"type": "function","function":{"name":"generate_password","description":"Generate a random password","parameters":{"type":"object","properties":{"length":{"type":"integer","description":"The length of the password"}},"required":["length"]}}}
       ]
    if not msgs or len(msgs) == 0:
        msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}]
    else:
        msgs.append({"role": "user", "content": user_query})

    res = chat_method(model="gpt-4-0125-preview", functions=functions, msgs=msgs)
    res_next = res
    if res_next.message.content and len(res_next.message.content) > 0:
        print("\n[AI response]:\n", res_next.message.content)
    else:
        print("\n[AI calling functions]:")
        for tool_call in res_next.message.tool_calls:
            print(f"Tool Call: {tool_call.function}")
    l = 0
    while res_next.message.tool_calls and len(res_next.message.tool_calls) > 0:
        msgs = insert_tool_response(res_next, msgs)
        res_next = chat_method(model="gpt-4-0125-preview", functions=functions, msgs=msgs)
        print(f"Loop {l}")
        if res_next.message.content and len(res_next.message.content) > 0:
            print("\n[AI response]:\n", res_next.message.content)
        else:
            print("\n[AI calling functions]:")
            for tool_call in res_next.message.tool_calls:
                print(f"Tool Call: {tool_call.function}")
        l += 1
    

## Multi + Parallel Function Call Test examples

In [44]:
import openai
local_api_key = "sk-"
local_base_url = "http://localhost:1234/v1/"

In [45]:

get_mistral_rubra_response = partial(get_oai_response, api_key=local_api_key, base_url=local_base_url)

user_query = "What is the distance between San Francisco and Cupertino by driving and by air from both directions?"
msgs = run_completion(get_mistral_rubra_response, user_query)



[AI calling functions]:
Tool Call: Function(arguments='{"origin":"San Francisco","destination":"Cupertino","mode":"driving"}', name='calculate_distance')
Observation: {'role': 'tool', 'tool_call_id': 'ee97d54b', 'name': 'calculate_distance', 'content': 'Distance is 50 miles.'}
Loop 0

[AI calling functions]:
Tool Call: Function(arguments='{"origin":"Cupertino","destination":"San Francisco","mode":"driving"}', name='calculate_distance')
Observation: {'role': 'tool', 'tool_call_id': 'f8fe0456', 'name': 'calculate_distance', 'content': 'Distance is 50 miles.'}
Loop 1

[AI calling functions]:
Tool Call: Function(arguments='{"origin":"San Francisco","destination":"Cupertino","mode":"air"}', name='calculate_distance')
Observation: {'role': 'tool', 'tool_call_id': '07513366', 'name': 'calculate_distance', 'content': 'Distance is 50 miles.'}
Loop 2

[AI calling functions]:
Tool Call: Function(arguments='{"origin":"Cupertino","destination":"San Francisco","mode":"air"}', name='calculate_distan

In [46]:
user_query0 = "create a 3d model with length 8, width 5, and height 4"
msgs = run_completion(get_mistral_rubra_response, user_query0)


[AI response]:
 It seems you've provided the dimensions for the 3D model, but the object name is missing. Please specify the name of the object you want to model along with the dimensions.


In [47]:
user_query2 = "now order 3 umbrellas for me and generate a password of length 8"
msgs = run_completion(get_mistral_rubra_response, user_query2, msgs)


[AI calling functions]:
Tool Call: Function(arguments='{"number_to_buy":3}', name='orderUmbrella')
Observation: {'role': 'tool', 'tool_call_id': '4db8834e', 'name': 'orderUmbrella', 'content': 'Order placed. the price is 10 dollars.'}
Loop 0

[AI calling functions]:
Tool Call: Function(arguments='{"length":8}', name='generate_password')
Observation: {'role': 'tool', 'tool_call_id': 'ef8b1e3d', 'name': 'generate_password', 'content': 'Password generated: 75893dd8'}
Loop 1

[AI response]:
 Your order for 3 umbrellas has been placed, and each umbrella costs $10. Additionally, a random password '75893dd8' of length 8 has been generated for you. Is there anything else you would like to do?


## Simple Math Chaining

In [48]:
user_query3 = "What is four plus six? What is the result of that plus 2? Take the result and multiply by 5 and then divide by two"


msgs = run_completion(get_mistral_rubra_response, user_query3)


[AI calling functions]:
Tool Call: Function(arguments='{"a":"4","b":"6"}', name='addition')
Observation: {'role': 'tool', 'name': 'addition', 'content': '10.0', 'tool_call_id': 'b8416100'}
Loop 0

[AI calling functions]:
Tool Call: Function(arguments='{"a":"10.0","b":"2"}', name='addition')
Observation: {'role': 'tool', 'name': 'addition', 'content': '12.0', 'tool_call_id': '55ffc78d'}
Loop 1

[AI calling functions]:
Tool Call: Function(arguments='{"a":"12.0","b":"5"}', name='multiplication')
Observation: {'role': 'tool', 'name': 'multiplication', 'content': '60.0', 'tool_call_id': '950ad86d'}
Loop 2

[AI calling functions]:
Tool Call: Function(arguments='{"a":"60.0","b":"2"}', name='division')
Observation: {'role': 'tool', 'name': 'division', 'content': '30.0', 'tool_call_id': '1f1d30f7'}
Loop 3

[AI response]:
 The result of four plus six is ten. Adding two to that gives twelve. Multiplying twelve by five gives sixty, and dividing sixty by two results in thirty.


# Condition

In [49]:
user_query4 = "check the weather in boston, calculate the distance from boston to NYC for me only if it's less than 100 degrees Fahrenheit"
msgs = run_completion(get_mistral_rubra_response, user_query4)


[AI calling functions]:
Tool Call: Function(arguments='{"location":"Boston","unit":"f"}', name='getCurrentWeather')

Observation: {'role': 'tool', 'tool_call_id': '8d94e07e', 'name': 'getCurrentWeather', 'content': 'temprature is 56 degree'}
Loop 0

[AI response]:
 The temperature in Boston is 56 degrees Fahrenheit. However, I cannot calculate the distance from Boston to NYC as the condition for doing so (temperature being less than 100 degrees Fahrenheit) is not met.


In [50]:
user_query5 = "check the weather in boston, calculate the distance from boston to NYC for me only if it's greater than 100 degrees Fahrenheit"
msgs = run_completion(get_mistral_rubra_response, user_query5)


[AI calling functions]:
Tool Call: Function(arguments='{"location":"Boston","unit":"f"}', name='getCurrentWeather')

Observation: {'role': 'tool', 'tool_call_id': 'ad770add', 'name': 'getCurrentWeather', 'content': 'temprature is 56 degree'}
Loop 0

[AI response]:
 The temperature in Boston is 56 degrees Fahrenheit. Since it's not greater than 100 degrees, I won't calculate the distance to NYC. Is there anything else you need?


# dependency

In [53]:
user_query6 = "check the size of all files in the 'documents' directory."
msgs = run_completion(get_mistral_rubra_response, user_query6)


[AI calling functions]:
Tool Call: Function(arguments='{"directory":"documents"}', name='list_files')
Observation: {'role': 'tool', 'tool_call_id': 'eea5d723', 'name': 'list_files', 'content': 'File list:\nreport.docx\ntask.txt\nnotes.txt'}
Loop 0

[AI calling functions]:
Tool Call: Function(arguments='{"filename":"report.docx"}', name='get_file_size')
Observation: {'role': 'tool', 'tool_call_id': 'c1408599', 'name': 'get_file_size', 'content': 'the size is 100 bytes.'}
Loop 1

[AI calling functions]:
Tool Call: Function(arguments='{"filename":"task.txt"}', name='get_file_size')
Observation: {'role': 'tool', 'tool_call_id': '44149b43', 'name': 'get_file_size', 'content': 'the size is 100 bytes.'}
Loop 2

[AI calling functions]:
Tool Call: Function(arguments='{"filename":"notes.txt"}', name='get_file_size')
Observation: {'role': 'tool', 'tool_call_id': '4ca922e8', 'name': 'get_file_size', 'content': 'the size is 100 bytes.'}
Loop 3

[AI response]:
 The sizes of the files in the 'docume